## Set Up

In [1]:
import os
import sys
import pandas as pd
import logging
from datetime import datetime, date
from google.cloud import bigquery
from typing import List


# **** SETUP ****

# change to match your filesystem
DATA_DIR = "../data/garys_store/files/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "receipts_1nf.csv")
# change to match your gcloud project 
PROJECT_NAME = "deb-01-372112"
DATASET_NAME = "tickets"
DATA_DIR = "./data/"
DEFAULT_TICKETS_FILE = os.path.join(DATA_DIR, "tickets.json")

# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'receipts': {
        'table_name': 'receipts',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED', max_length=128),
            bigquery.SchemaField('confirmation', 'string', mode='REQUIRED'),
            bigquery.SchemaField('ticket_date', 'date', mode='REQUIRED'),
            bigquery.SchemaField('price', 'float', mode='REQUIRED'),
            bigquery.SchemaField('seat', 'string', mode='REQUIRED'),
            bigquery.SchemaField('status', 'string', mode='REQUIRED'),
        ]
    }
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-15 09:47:06,455][2821305102:0051] : Creating bigquery client
[INFO ][2023-01-15 09:47:06,463][2821305102:0054] : Setup Completed


## Load Original Ticket File

In [12]:
import json 
# receipts data file name
filename = DEFAULT_TICKETS_FILE
logger.debug(f"attempting to process: {filename}")

# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
with open('./data/tickets.json') as data_file:    
    data = [json.loads(line) for line in data_file]

df = pd.json_normalize(data)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# *** always perform check at the end ***
# check schema: contains all expected columns?
expected_columns = ['eticket_num', 'confirmation', 'ticket_date', 'price', 'seat', 'status', 'origin', 'destination']
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

# assign & remember tickets dataframe
tickets_df = df
display(tickets_df.head(n=10))

[DEBUG][2023-01-15 10:02:59,561][2291644546:0004] : attempting to process: ./data/tickets.json
[INFO ][2023-01-15 10:03:00,011][2291644546:0017] : loaded 4096 rows from: ./data/tickets.json


,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline.iata,airline.icao,airline.callsign,...,passenger.last_name,passenger.gender,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Hawaiian Airlines,HA,HAL,HAWAIIAN,...,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,NaN,NaN
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Wizz Air,W6,WZZ,WIZZ AIR,...,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093,NaN,NaN
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,AirAsia,AK,AXM,ASIAN EXPRESS,...,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,NaN,NaN
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Xiamen Airlines,MF,CXA,XIAMEN AIR,...,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,NaN,NaN
5,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,Air New Zealand,NZ,ANZ,NEW ZEALAND,...,Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,NaN,NaN
6,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,Jeju Air,7C,JJA,JEJU AIR,...,Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,NaN,NaN
7,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,American Airlines,AA,AAL,AMERICAN,...,Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN
8,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,...,Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,NaN,NaN
9,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,...,Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,NaN,NaN


## Look up airlines.iata, airports.iata, passenger.passenger_sk

In [22]:
import db_dtypes
# products table query
query = f"""
SELECT
  iata,
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.airlines`
"""
# query bigquery table into dataframe
aldf = client.query(query).to_dataframe()
# index by product_name for the join
aldf = aldf.set_index(keys='iata')
logger.info(f"query {len(aldf.index)} rows from airlines table")

# customers table query
query = f"""
SELECT
  iata
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.airports`
"""
# query bigquery table into dataframe
apdf = client.query(query).to_dataframe()
# index by full_name for the join
apdf = apdf.set_index(keys='iata')
logger.info(f"query {len(apdf.index)} rows from airports table")

# customers table query
query = f"""
SELECT
  passenger_sk,
  email
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.passengers`
"""
# query bigquery table into dataframe
pdf = client.query(query).to_dataframe()
# index by full_name for the join
pdf = pdf.set_index(keys='passenger_sk')
logger.info(f"query {len(pdf.index)} rows from passengers table")

# drop email if they already exist
#   this line will allow us to re-run this cell
df = df.drop(columns=['email_x', 'email_y'], errors='ignore')

logger.debug(f"preparing for join")
# join with the main dataframe to get product_id and customer_id
df = df.merge(aldf, left_on='airline.iata', right_on='iata', how='inner')
df = df.merge(apdf, left_on='origin.iata', right_on='iata', how='inner')
df = df.merge(pdf, left_on='passenger.email', right_on='email', how='inner')

logger.debug(f"dataframe:")
display(df.head(10))


[INFO ][2023-01-15 10:28:39,769][1041266006:0013] : query 48 rows from airlines table
[INFO ][2023-01-15 10:28:41,561][1041266006:0026] : query 386 rows from airports table
[INFO ][2023-01-15 10:28:43,441][1041266006:0040] : query 32 rows from passengers table
[DEBUG][2023-01-15 10:28:43,482][1041266006:0046] : preparing for join
[DEBUG][2023-01-15 10:28:43,578][1041266006:0052] : dataframe:


,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline.iata,airline.icao,airline.callsign,...,passenger.gender,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination,email
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
1,969-103858-4329,H6EFSS,2022-03-22,618.11,22F,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
2,325-715930-9053,NU9YWL,2022-03-24,535.64,16A,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
3,883-113178-7964,2IANI2,2022-03-23,898.10,4H,active,JetBlue Airways,B6,JBU,JETBLUE,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
4,362-056643-6144,G29QFS,2022-03-22,827.27,9I,active,Air China,CA,CCA,AIR CHINA,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
5,931-758358-3390,6LAIS9,2022-03-24,780.59,29G,active,Vietnam Airlines,VN,HVN,VIET NAM AIRLINES,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
6,038-106382-6573,MCCY20,2022-03-21,743.85,30D,active,Shenzhen Airlines,ZH,CSZ,SHENZHEN AIR,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
7,808-619797-0756,V8IWMD,2022-03-23,358.24,1E,active,Vietnam Airlines,VN,HVN,VIET NAM AIRLINES,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
8,071-020974-0594,Z3IM2B,2022-03-24,339.62,14G,active,Cape Air,9K,KAP,CAIR,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com
9,904-530679-1980,V9XA2I,2022-03-24,442.82,16E,active,Southwest Airlines,WN,SWA,SOUTHWEST,...,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN,robert.brown.69@hotmail.com


## 

In [ ]:
'destination.name', 'destination.city', 'destination.country', 'destination.iata', 'destination.icao', 'destination.latitude', 'destination.longitude', 'destination.altitude', 'destination.tz_timezone']